In [1]:
import requests
from bs4 import BeautifulSoup

def get_hub_info(url, class_to_hub_areas):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, 'html.parser')
    course_cards = soup.find_all(class_='cf-course-card')

    for course_card in course_cards:
        class_id_span = course_card.find(class_='cf-course-id')
        if class_id_span:
            class_name = ' '.join(class_id_span.get_text().split())
            hub_area_lis = course_card.select('.cf-hub-ind .cf-hub-offerings li')
            hub_areas = [li.get_text() for li in hub_area_lis]
            class_to_hub_areas[class_name] = hub_areas

class_to_hub_areas = {}

urls = [
    'https://www.bu.edu/hub/hub-courses/philosophical-aesthetic-and-historical-interpretation/',
    'https://www.bu.edu/hub/hub-courses/scientific-and-social-inquiry/',
    'https://www.bu.edu/hub/hub-courses/quantitative-reasoning/',
    'https://www.bu.edu/hub/hub-courses/diversity-civic-engagement-and-global-citizenship/',
    'https://www.bu.edu/hub/hub-courses/communication/',
    'https://www.bu.edu/hub/hub-courses/intellectual-toolkit/'
]

for url in urls:
    get_hub_info(url, class_to_hub_areas)

#bad course
del class_to_hub_areas['CAS MS 502']
del class_to_hub_areas['CAS LS 309']

In [2]:
from bs4 import BeautifulSoup
import requests

major = ['CDS DS 110', 'CDS DS 120', 'CDS DS 121', 'CDS DS 122', 'CDS DS 210', 'CDS DS 310', 'CAS MA 214', 'CDS DS 320', 'CDS DS 340', 'CDS DS 380', 'CAS MA 415', 'CAS CS 506', 'QST BA 476', 'QST BA 305', 'CDS DS 549', 'CDS DS 100', 'CAS WR 120', 'CAS WR 152']

req = {}


for course in major:
    college, sub, num = course.split(' ')

    url = f'https://www.bu.edu/phpbin/course-search/search.php?page=w0&pagesize=10&adv=1&nolog=&search_adv_all={college}+{sub}+{num}&yearsem_adv=2024SPRG&credits=*&pathway=&hub_match=all'

    html_content = requests.get(url).text

    soup = BeautifulSoup(html_content, 'html.parser')
    hub_list_ul = soup.find('ul', class_='coursearch-result-hub-list')

    if hub_list_ul:
        hub_list = [li.get_text().strip() for li in hub_list_ul.find_all('li') if li.get_text()]
        req[course] = hub_list
    else:
        req[course] = []


In [3]:
hub_requirements = {'Philosophical Inquiry and Life’s Meanings': 1, 'Aesthetic Exploration': 1, 'Historical Consciousness': 1, 'Scientific Inquiry I': 1,
                    'Social Inquiry I': 1, 'Social Inquiry II': 1, 'Quantitative Reasoning I': 1, 'Quantitative Reasoning II': 1,
                    'The Individual in Community': 1, 'Global Citizenship and Intercultural Literacy': 2, 'Ethical Reasoning': 1, 'First-Year Writing Seminar': 1,
                    'Writing, Research, and Inquiry': 1, 'Writing-Intensive Course': 2, 'Oral and/or Signed Communication': 1, 'Digital/Multimedia Expression': 1,
                    'Critical Thinking': 2, 'Research and Information Literacy': 2, 'Teamwork/Collaboration': 2, 'Creativity/Innovation': 2}
#REMOVED SCIENTIFIC INQUIRY II

In [4]:
def optimize_with_major_classes(major, hub_requirements, req, class_to_hub_areas):
    selected_classes = major[:]
    unmet_requirements = hub_requirements.copy()

    for course in major:
        if course in req:
            for hub in req[course]:
                if hub in unmet_requirements and unmet_requirements[hub] > 0:
                    unmet_requirements[hub] -= 1

    def class_utility(class_hubs):
        return sum(1 for hub in class_hubs if hub in unmet_requirements and unmet_requirements[hub] > 0)

    while any(count > 0 for count in unmet_requirements.values()):
        combined_classes = {**{k: v for k, v in req.items() if k not in major}, **class_to_hub_areas}
        best_class = max(combined_classes.items(), key=lambda x: class_utility(x[1]))

        if class_utility(best_class[1]) == 0:
            break

        selected_classes.append(best_class[0])
        for hub in best_class[1]:
            if hub in unmet_requirements and unmet_requirements[hub] > 0:
                unmet_requirements[hub] -= 1

        if best_class[0] in req:
            del req[best_class[0]]
        else:
            del class_to_hub_areas[best_class[0]]

    return selected_classes

optimized_classes_with_major = optimize_with_major_classes(major, hub_requirements, req, class_to_hub_areas)

print("Optimized list of classes to fulfill all hub requirements:")
for course in optimized_classes_with_major:
    print(course)

'''
1st
CDS DS 110
CDS DS 121
CDS DS 100
CAS WR 120

2nd
CDS DS 121
CDS DS 210
CAS MA 214
CAS WR 152


3rd
CDS DS 122
CDS DS 310
CDS DS 415
CAS AH 331


4th
CDS DS 320
CAS CS 506
QST BA 305
CAS AN 235


5th
CDS DS 340
CDS DS 380
QST BA 476
CAS JS 348
CAS AN 250

6th
CDS DS 549

7th

8th
'''

Optimized list of classes to fulfill all hub requirements:
CDS DS 110
CDS DS 120
CDS DS 121
CDS DS 122
CDS DS 210
CDS DS 310
CAS MA 214
CDS DS 320
CDS DS 340
CDS DS 380
CAS MA 415
CAS CS 506
QST BA 476
QST BA 305
CDS DS 549
CDS DS 100
CAS WR 120
CAS WR 152
CAS AH 331
CAS AN 235
CAS JS 348
CAS AN 250


'\n1st\nCDS DS 110\nCDS DS 121\nCDS DS 100\nCAS WR 120\n\n2nd\nCDS DS 121\nCDS DS 210\nCAS MA 214\nCAS WR 152\n\n\n3rd\nCDS DS 122\nCDS DS 310\nCDS DS 415\n\n\n\n4th\nCDS DS 320\nCAS CS 506\nQST BA 305\n\n\n5th\nCDS DS 340\nCDS DS 380\nQST BA 476\n\n6th\nCDS DS 549\n\n7th\n\n8th\n'

In [5]:
def checker(req, class_to_hub_areas, optimized_classes_with_major, hub_requirements):
    sanitized_hub_requirements = {key.strip().lower(): value for key, value in hub_requirements.items()}
    hubs_tally = {key: 0 for key in sanitized_hub_requirements}

    hub_classes = [key for key in optimized_classes_with_major if key not in req]

    for class_name in hub_classes:
        if class_name in class_to_hub_areas:
            sanitized_hubs = [hub.strip().lower() for hub in class_to_hub_areas[class_name]]
            req[class_name] = sanitized_hubs

    for class_name, hubs in req.items():
        for hub in hubs:
            sanitized_hub = hub.strip().lower()
            if sanitized_hub in hubs_tally:
                hubs_tally[sanitized_hub] += 1

    for key, value in sanitized_hub_requirements.items():
        if hubs_tally.get(key, 0) >= value:
            print(f"Requirement '{key}' met: {hubs_tally.get(key, 0)} / {value}")

checker(req, class_to_hub_areas, optimized_classes_with_major, hub_requirements)


Requirement 'social inquiry i' met: 1 / 1
Requirement 'social inquiry ii' met: 1 / 1
Requirement 'quantitative reasoning i' met: 3 / 1
Requirement 'quantitative reasoning ii' met: 6 / 1
Requirement 'ethical reasoning' met: 3 / 1
Requirement 'first-year writing seminar' met: 1 / 1
Requirement 'writing, research, and inquiry' met: 1 / 1
Requirement 'digital/multimedia expression' met: 4 / 1
Requirement 'critical thinking' met: 7 / 2
Requirement 'research and information literacy' met: 5 / 2
Requirement 'teamwork/collaboration' met: 4 / 2
